In [1]:
!pip -q install folium
!pip -q install geopy
!pip -q install geocoder



In [3]:
import numpy as np 

import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json 

from geopy.geocoders import Nominatim
import geocoder 

import requests 
from bs4 import BeautifulSoup 

from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

In [4]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Chennai").text

In [5]:
soup = BeautifulSoup(data, 'html.parser')

In [6]:
neighborhoodList = []

In [7]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
chennai_df = pd.DataFrame({"Neighborhood": neighborhoodList})

chennai_df.head()

,Neighborhood
0,Alandur
1,Anna Nagar
2,"Ashok Nagar, Chennai"
3,Assisi Nagar
4,Ayanavaram


In [9]:
chennai_df.shape

(60, 1)

In [10]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
coords = [ get_latlng(neighborhood) for neighborhood in chennai_df["Neighborhood"].tolist() ]

In [12]:
coords

[[13.000100000000032, 80.20048000000008],
 [13.083590000000072, 80.21014000000008],
 [13.035280000000057, 80.21218000000005],
 [13.16453000000007, 80.23279000000008],
 [13.09883000000002, 80.23238000000003],
 [13.047083182901506, 80.22143009437926],
 [12.932770000000062, 80.14388000000008],
 [12.95234000000005, 80.14411000000007],
 [12.988600000000076, 80.15099000000004],
 [12.827270000000055, 80.22871000000004],
 [13.04051000000004, 80.13650000000007],
 [13.110770000000059, 80.21309000000008],
 [13.129720000000077, 80.18300000000005],
 [13.10728000000006, 80.15354000000008],
 [12.956160000000068, 80.17874000000006],
 [12.793440000000032, 80.22011000000003],
 [13.081990000000076, 80.24448000000007],
 [13.051470000000052, 80.22428000000008],
 [13.13662000000005, 80.24477000000007],
 [13.141423609374849, 80.19409121688952],
 [13.09617000000003, 80.05290000000008],
 [13.116800000000069, 80.27730000000008],
 [13.183260000000075, 80.24059000000005],
 [13.157550000000072, 80.24282000000005],

In [13]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [14]:
chennai_df['Latitude'] = df_coords['Latitude']
chennai_df['Longitude'] = df_coords['Longitude']

In [15]:
print(chennai_df.shape)
chennai_df

(60, 3)


,Neighborhood,Latitude,Longitude
0,Alandur,13.000100,80.200480
1,Anna Nagar,13.083590,80.210140
2,"Ashok Nagar, Chennai",13.035280,80.212180
3,Assisi Nagar,13.164530,80.232790
4,Ayanavaram,13.098830,80.232380
5,Chennai city,13.047083,80.221430
6,Chitlapakkam,12.932770,80.143880
7,Chromepet,12.952340,80.144110
8,Cowl Bazaar,12.988600,80.150990
9,Egattur (Kanchipuram District),12.827270,80.228710


In [16]:
address = 'Chennai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai, India 13.0801721, 80.2838331.


In [30]:

# create map of Chennai using latitude and longitude values
map_chennai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(chennai_df['Latitude'], chennai_df['Longitude'], chennai_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_chennai)  
    
map_chennai

In [20]:
map_chennai.save('map_chennai.html')

In [26]:

# define Foursquare Credentials and Version
CLIENT_ID = 'GY0RQ4L2WFGGZZU50QN5X0ICXO4C3QFUR1TDSCZTFRCDG3UT' # your Foursquare ID
CLIENT_SECRET = 'W3XU0NT5UGCZVPOF5PHQSY1MFDFM0KEOGIRX5H0KCILIJGIS' # your Foursquare Secret
VERSION = '20190913' # Foursquare API version

In [27]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(chennai_df['Latitude'], chennai_df['Longitude'], chennai_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [31]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1163, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alandur,13.0001,80.20048,Sukkkubai Beef Biryani Shop,12.998769,80.201381,Indian Restaurant
1,Alandur,13.0001,80.20048,Aasife & Brothers Biryani Centre,13.000457,80.200635,Indian Restaurant
2,Alandur,13.0001,80.20048,Q Bar,13.016606,80.204853,Restaurant
3,Alandur,13.0001,80.20048,Hilton Chennai,13.016621,80.204787,Hotel
4,Alandur,13.0001,80.20048,Moon & Six Pence - The Irish Bar,13.007848,80.208152,Bar


In [32]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alandur,42,42,42,42,42,42
Anna Nagar,100,100,100,100,100,100
"Ashok Nagar, Chennai",71,71,71,71,71,71
Assisi Nagar,5,5,5,5,5,5
Ayanavaram,32,32,32,32,32,32
Chennai city,100,100,100,100,100,100
Chitlapakkam,14,14,14,14,14,14
Chromepet,18,18,18,18,18,18
Cowl Bazaar,19,19,19,19,19,19


In [33]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 145 uniques categories.


In [34]:
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Restaurant', 'Hotel', 'Bar', 'Pizza Place',
       'Asian Restaurant', 'Train Station', 'Ice Cream Shop',
       'South Indian Restaurant', 'Donut Shop', 'Church',
       'Breakfast Spot', 'Pool Hall', 'Japanese Restaurant',
       'Multicuisine Indian Restaurant', 'Bakery', 'Hotel Bar', 'Café',
       'Juice Bar', 'Fast Food Restaurant', 'Department Store',
       'Cafeteria', 'Gym Pool', 'Sandwich Place', 'Park', 'Gym',
       'Coffee Shop', 'Snack Place', 'Chinese Restaurant',
       'Vegetarian / Vegan Restaurant', 'Indian Sweet Shop', 'Shoe Store',
       'Middle Eastern Restaurant', 'Burger Joint', 'American Restaurant',
       'Shopping Mall', 'Italian Restaurant', 'Farmers Market', 'Market',
       'Paper / Office Supplies Store', 'Multiplex', 'Clothing Store',
       'Jewelry Store', 'BBQ Joint', 'Furniture / Home Store',
       'New American Restaurant', 'Bookstore', 'Electronics Store',
       'Scenic Lookout', 'Mobile Phone Shop'], dtype=object)

In [35]:
"Gym" in venues_df['VenueCategory'].unique()

True

In [37]:

# one hot encoding
chennai_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chennai_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [chennai_onehot.columns[-1]] + list(chennai_onehot.columns[:-1])
chennai_onehot = chennai_onehot[fixed_columns]

print(chennai_onehot.shape)
chennai_onehot.head()

(1163, 146)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Arcade,Asian Restaurant,BBQ Joint,Badminton Court,Bakery,Bar,Beach,Beer Garden,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Canal,Chinese Restaurant,Church,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Coworking Space,Cupcake Shop,Daycare,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym Pool,Harbor / Marina,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,National Park,New American Restaurant,Nightclub,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Racetrack,Recreation Center,Resort,Restaurant,River,Road,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Supermarket,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:

chennai_grouped = chennai_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(chennai_grouped.shape)
chennai_grouped

(60, 146)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Antique Shop,Arcade,Asian Restaurant,BBQ Joint,Badminton Court,Bakery,Bar,Beach,Beer Garden,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Campground,Canal,Chinese Restaurant,Church,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Coworking Space,Cupcake Shop,Daycare,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Field,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym Pool,Harbor / Marina,Hospital,Hotel,Hotel Bar,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Men's Store,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,National Park,New American Restaurant,Nightclub,Paper / Office Supplies Store,Park,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Racetrack,Recreation Center,Resort,Restaurant,River,Road,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,Soccer Stadium,South Indian Restaurant,Spa,Sporting Goods Shop,Supermarket,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Alandur,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.023810,0.000000,0.000000,0.023810,0.023810,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.02381,0.047619,0.00,0.0,0.000000,0.02381,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.023810,0.000000,0.0,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.0,0.119048,0.02381,0.000000,0.000000,0.023810,0.214286,0.00,0.0000,0.000000,0.000000,0.02381,0.00,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0,0.023810,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.000000,0.047619,0.000000,0.000000,0.023810,0.000000,0.000000,0.00,0.000000,0.047619,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.071429,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Anna Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.010000,0.010000,0.000000,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.00,0.000000,0.010000,0.010000,0.030000,0.000000,0.000000,0.00000,0.040000,0.00,0.0,0.030000,0.00000,0.040000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.0,0.010000,0.120000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.02,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.0,0.010000,0.01000,0.000000,0.000000,0.040000,0.140000,0.01,0.0000,0.000000,0.010000,0.00000,0.01,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.020000,0.00,0.01,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.030000,0.00

In [42]:
len(chennai_grouped[chennai_grouped["Gym"] > 0])

6

In [43]:

chennai_gym = chennai_grouped[["Neighborhoods","Gym"]]

In [44]:
chennai_gym.head()

,Neighborhoods,Gym
0,Alandur,0.00
1,Anna Nagar,0.01
2,"Ashok Nagar, Chennai",0.00
3,Assisi Nagar,0.00
4,Ayanavaram,0.00


In [45]:
kclusters = 3

chennai_clustering = chennai_gym.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chennai_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [47]:
chennai_merged = chennai_gym.copy()

# add clustering labels
chennai_merged["Cluster Labels"] = kmeans.labels_

In [48]:
chennai_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
chennai_merged.head()

,Neighborhood,Gym,Cluster Labels
0,Alandur,0.00,0
1,Anna Nagar,0.01,0
2,"Ashok Nagar, Chennai",0.00,0
3,Assisi Nagar,0.00,0
4,Ayanavaram,0.00,0


In [49]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
chennai_merged = chennai_merged.join(chennai_df.set_index("Neighborhood"), on="Neighborhood")

print(chennai_merged.shape)
chennai_merged.head() # check the last columns!

(60, 5)


,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Alandur,0.00,0,13.00010,80.20048
1,Anna Nagar,0.01,0,13.08359,80.21014
2,"Ashok Nagar, Chennai",0.00,0,13.03528,80.21218
3,Assisi Nagar,0.00,0,13.16453,80.23279
4,Ayanavaram,0.00,0,13.09883,80.23238


In [50]:

# sort the results by Cluster Labels
print(chennai_merged.shape)
chennai_merged.sort_values(["Cluster Labels"], inplace=True)
chennai_merged

(60, 5)


,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Alandur,0.000000,0,13.000100,80.200480
31,Navalur,0.000000,0,12.845870,80.226460
32,Nazarethpettai,0.000000,0,13.037110,80.057510
33,Oragadam,0.000000,0,13.157150,80.138320
34,Padappai,0.000000,0,12.886500,80.021550
35,Pallavaram,0.000000,0,12.974330,80.148430
36,Pallikaranai,0.000000,0,12.954890,80.222960
37,Panambakkam,0.000000,0,13.083620,80.282520
38,Pattabiram,0.000000,0,13.123300,80.059390
39,Perambakkam,0.000000,0,12.905590,80.209000


In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chennai_merged['Latitude'], chennai_merged['Longitude'], chennai_merged['Neighborhood'], chennai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [53]:
chennai_merged.loc[chennai_merged['Cluster Labels'] == 0]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
0,Alandur,0.000000,0,13.000100,80.200480
31,Navalur,0.000000,0,12.845870,80.226460
32,Nazarethpettai,0.000000,0,13.037110,80.057510
33,Oragadam,0.000000,0,13.157150,80.138320
34,Padappai,0.000000,0,12.886500,80.021550
35,Pallavaram,0.000000,0,12.974330,80.148430
36,Pallikaranai,0.000000,0,12.954890,80.222960
37,Panambakkam,0.000000,0,13.083620,80.282520
38,Pattabiram,0.000000,0,13.123300,80.059390
39,Perambakkam,0.000000,0,12.905590,80.209000


In [54]:
chennai_merged.loc[chennai_merged['Cluster Labels'] == 1]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
45,Puzhal,0.333333,1,13.15947,80.20715


In [55]:
chennai_merged.loc[chennai_merged['Cluster Labels'] == 2]

,Neighborhood,Gym,Cluster Labels,Latitude,Longitude
55,Thuraipakkam,0.083333,2,12.90264,80.1843


We can see from the above most of the gyms are concentrated in Thuraipakkam that is the cluster 2. Cluster 0 is very less number of the gym and Cluster 1 is having the highest number of Gym.